In [1]:
import nltk
from nltk.tag import str2tuple
from nltk.tag.hmm import HiddenMarkovModelTrainer, HiddenMarkovModelTagger
import numpy as np
from pyvi import ViTokenizer
from nltk.tag.util import untag

In [2]:
file_tranning = 'vi_train.pos'
file_testing = 'vi_test.pos'

train_set = []

with open(file_tranning, encoding='UTF-8') as f:
    train_lines = f.readlines()
    for line in train_lines:
        if(line.strip()): # non-empty line
            tmp = []
            for word in line.split():
                word_tag = nltk.tag.str2tuple(word)
                tmp.append(word_tag)
            train_set.append(tmp)

In [3]:
# print(train_set[0])

In [4]:
# trainer = HiddenMarkovModelTrainer()
# hmm = trainer.train(train_set)

hmm = HiddenMarkovModelTagger.train(train_set)

In [5]:
def pos_result(sentences):
    str = ""
    for word, tag in hmm.tag(sentences.split()):
#         print(word)
#         print(tag)
        str += word + "/" + tag + " "
    return str.rstrip() # remove whitespace from the end of string

### Input vào một câu tiếng Việt, cho ra nhãn của từng từ trong câu

In [6]:
input_sentences = "Dù khá đắt nhưng tôi vẫn đồng ý."
print(pos_result(ViTokenizer.tokenize(input_sentences))) # Word Segmentation 

Dù/C khá/R đắt/A nhưng/C tôi/P vẫn/R đồng_ý/V ./.


In [7]:
# test_text = "Anh Anh bảo : “ Đồ_đạc ướt thì đem phơi được , chớ bà_con thì làm răng ... ” ."
# print(pos_result(ViTokenizer.tokenize(test_text))) 

In [8]:
with open(file_testing, encoding="UTF-8") as f:
    test_lines = [line for line in f.readlines() if line.strip()] # Remove empty lines
print(len(test_lines))

1051


In [9]:
# print(test_lines[0])

In [10]:
test_data = np.array(test_lines)
test_sentences = [sent for sent in np.char.split(test_data)]
print(test_sentences[0])

['Trên/E', 'đường/N', 'xuất_hiện/V', 'nhiều/A', 'cặp/N', 'cha/N', '-/-', 'con/N', ',/,', 'mẹ/N', '-/-', 'con/N', ',/,', 'hay/C', 'có_khi/X', 'là/C', 'cả/T', 'nhà/N', ',/,', 'tay_xách_nách_mang/X', 'vừa/R', 'đi/V', 'vừa/R', 'dò/V', 'bản_đồ/N', './.']


In [11]:
# xử lý test bỏ hết tag
test_sent = []

for sent in test_sentences:
    word = []
    for word_tag in sent:
        tmp = nltk.tag.str2tuple(word_tag)
        word.append(tmp[0])
    word = " ".join(word)
    test_sent.append(word)
print(test_sent[0])

Trên đường xuất_hiện nhiều cặp cha - con , mẹ - con , hay có_khi là cả nhà , tay_xách_nách_mang vừa đi vừa dò bản_đồ .


In [12]:
pred_lines = []
for i in range(len(test_lines)):
    pred_lines.append(pos_result(test_sent[i]))

In [13]:
# print(pred_lines)

In [14]:
total_word_in_test = 0
total_word_in_pred = 0

test = []
for i in range(len(test_lines)):
    for j in test_lines[i].split():
        test.append(j)
        total_word_in_test += 1
                
pred = []
for i in range(len(pred_lines)):
    for j in pred_lines[i].split():
        pred.append(j)
        total_word_in_pred += 1

In [15]:
print(total_word_in_test)
print(total_word_in_pred)

23280
23280


In [16]:
# chuyển nhãn viết hoa
test1 = []
for j in test:
    x = nltk.tag.str2tuple(j)
    z = nltk.tag.tuple2str(x)
    test1.append(z)
# print(test1)

In [17]:
count_true_pred = 0
for i in range(23280):
    if(pred[i] == test1[i]):
        count_true_pred += 1
        
print(count_true_pred)

20966


### Đánh giá độ chính xác trên tập Test: tỉ lệ số từ gán nhãn đúng trên tổng số từ

In [18]:
print("Tỉ lệ số từ gán nhãn đúng: %.5f%%" % ((count_true_pred / total_word_in_test) * 100))

Tỉ lệ số từ gán nhãn đúng: 90.06014%


### Sử dụng thêm phương pháp phân loại:
Phân loại: sử dụng Maximum Entropy Model (Logistic Regression), SVM.

In [19]:
# from sklearn.linear_model import LogisticRegression

In [20]:
# model_lr = LogisticRegression(random_state=42)
# model_lr.fit()

In [21]:
# from sklearn import svm

In [22]:
# model_svm = svm.SVC(kernel='linear',gamma=0.01,random_state=0, tol=1e-5, C=100.)
# # model = svm.LinearSVC(random_state=0, tol=1e-5)
# model_svm.fit(X_train, y_train)

### Trình bày lý thuyết về Conditional Random Fields và sử dụng thư viện (hoặc open source) để giải quyết bài toán trên. 

In [23]:
tag_set = {'N', 'NP', 'NC', 'NU', 'V', 'A', 'P', 'L', 'M', 'R', 'E', 'C', 'I', 'T', 'B', 'Y', 'S', 'X', 'NY', 'NB', 'AB', 'VB', 'VY', 'AP'}

# Bỏ dấu câu trong tập train
train_set_not_punctuation = []

for sentences in train_set:
    tmp = []
    for word_tag in sentences:
        if(word_tag[1] in tag_set):
            tmp.append(word_tag)
    train_set_not_punctuation.append(tmp)
# print(train_set_not_punctuation[155])

In [24]:
# print(train_set[0])
# print("Tagged sentences: ", len(train_set))
# tagged_words=[tup for sent in train_set for tup in sent]
# print("Tagged words:", len(tagged_words))

In [25]:
print(train_set_not_punctuation[0])
print("Tagged sentences: ", len(train_set_not_punctuation))
tagged_words = [x for sent in train_set_not_punctuation for x in sent]
print("Tagged words:", len(tagged_words))

[('Trên', 'E'), ('đường', 'N'), ('đi', 'V'), ('có', 'V'), ('một', 'M'), ('lần', 'N'), ('xe', 'N'), ('cô', 'N'), ('suýt', 'R'), ('rơi', 'V'), ('xuống', 'R'), ('vực', 'N'), ('ở', 'E'), ('đèo', 'N'), ('Ngoạn_Mục', 'NP')]
Tagged sentences:  8445
Tagged words: 165281


In [26]:
# Bỏ dấu câu trong tập test
test_set_not_punctuation = []

for sentences in test_sentences:
    tmp = []
    for word_tag in sentences:
        x = nltk.tag.str2tuple(word_tag)
        if(x[1] in tag_set):
            tmp.append(x)
    test_set_not_punctuation.append(tmp)
# print(test_set_not_punctuation[0])

In [27]:
print(test_set_not_punctuation[0])
print("Tagged sentences: ", len(test_set_not_punctuation))
test_tagged_words = [x for sent in test_set_not_punctuation for x in sent]
print("Tagged words:", len(test_tagged_words))

[('Trên', 'E'), ('đường', 'N'), ('xuất_hiện', 'V'), ('nhiều', 'A'), ('cặp', 'N'), ('cha', 'N'), ('con', 'N'), ('mẹ', 'N'), ('con', 'N'), ('hay', 'C'), ('có_khi', 'X'), ('là', 'C'), ('cả', 'T'), ('nhà', 'N'), ('tay_xách_nách_mang', 'X'), ('vừa', 'R'), ('đi', 'V'), ('vừa', 'R'), ('dò', 'V'), ('bản_đồ', 'N')]
Tagged sentences:  1051
Tagged words: 19822


In [28]:
def func_features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    try: features = {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]  
         }
    except IndexError: features = {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': False,
        'is_all_caps': False,
        'is_all_lower': True,
        'prefix-1': '',
        'prefix-2': '',
        'prefix-3': '',
        'suffix-1': '',
        'suffix-2': '',
        'suffix-3': '',
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': False  
         }
    return features

def transform_to_dataset(tagged_sentences):
    X, y = [], []
    for tagged in tagged_sentences:
        X.append([func_features(untag(tagged), index) for index in range(len(tagged))])
        y.append([tag for _, tag in tagged])
    return X, y

X_train, y_train = transform_to_dataset(train_set_not_punctuation)
X_test, y_test = transform_to_dataset(test_set_not_punctuation)

print(len(X_train))
print(len(X_test))
# print(X_train[155])
# print(y_train[0])

8445
1051


In [29]:
from sklearn_crfsuite import CRF
 
model = CRF()
model.fit(X_train, y_train)

D:\Program\anaconda3\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(keep_tempfiles=None)

In [30]:
sentence = "Cô Hảo bây giờ sống qua ngày trong căn nhà gỗ từ đời ông để lại."

def pos_tag(sentence):
    word_segment = ViTokenizer.tokenize(sentence)
    list_sentence = list(word_segment.split(" "))
    
    sentence_features = [func_features(list_sentence, index) for index in range(len(list_sentence))]
    str = ""
    aaa = model.predict([sentence_features])

    for i in range(len(list_sentence)):
        str += list_sentence[i] + "/" + aaa[0][i] + " "
        
    return str.rstrip()

print(pos_tag(sentence))

Cô/NC Hảo/NP bây_giờ/P sống/V qua_ngày/N trong/E căn/NC nhà/N gỗ/N từ/E đời/N ông/N để/V lại/R ./NY


In [31]:
print("Cô/Nc Hảo/Np bây_giờ/P sống/V qua_ngày/X trong/E căn/Nc nhà/N gỗ/N từ/E đời/N ông/N để/V lại/R ./.")

Cô/Nc Hảo/Np bây_giờ/P sống/V qua_ngày/X trong/E căn/Nc nhà/N gỗ/N từ/E đời/N ông/N để/V lại/R ./.


In [32]:
from sklearn_crfsuite import metrics
 
y_pred = model.predict(X_test)
print("Accuracy score: %.5f%%" % (metrics.flat_accuracy_score(y_test, y_pred) * 100))

Accuracy score: 91.31773%
